首先截取 1900年以后的信息, 因为这样不用处理空值
通过每个国家地区的月均值与月方差, 每个月各生成10000条数据, 然后计算年度均值

In [ ]:
import pandas as pd 
import numpy as np 
import random 


# 设置随机种子
def seed_everything(SEED):
    np.random.seed(SEED)
    random.seed(SEED)
SEED = 42 
seed_everything(SEED)

# 读取数据
df = pd.read_csv('./2022_APMCM_C_Data.csv', encoding='gbk')

# dt转换为标准的时间序列格式
df.index = pd.DatetimeIndex(df['dt'])
df.drop(columns=['dt'], inplace=True)

# 纬度的处理
df['Latitude'] = df['Latitude'].map(lambda x: float(x[: -1]) if x[-1] == 'N' else -float(x[: -1]))
# 经度的处理
df['Longitude'] = df['Longitude'].map(lambda x: float(x[: -1]) if x[-1] == 'E' else 360 - float(x[: -1]))

# 生成单个位置
df['location'] = df['Country'] + ' ' + df['City']
df.drop(columns=['Country', 'City'], inplace=True)

df['year'] = df.index.strftime('%Y').astype(int)
df['month'] = df.index.strftime('%m').astype(int)

In [ ]:
df_1 = df.loc['1900': ]
df = df[df.index != '2013-09-01']
df_year_dict = {}
i = 1
for location in df_1['location'].unique():
    df_location = df_1[df_1['location']==location]
    location_dict = {}
    for year in df_location['year'].unique():
        if year == 2013:
            continue
        df_year = df_location[df_location['year']==year]
        temperature = []
        for month in df_year['month'].unique():
            mean = df_year.loc[df_year['month']==month, 'AverageTemperature'].values[0]
            std = df_year.loc[df_year['month']==month, 'AverageTemperatureUncertainty'].values[0]
            random_temperature = np.random.normal(mean, std, 10000)
            temperature.append(random_temperature)
        temperature = np.concatenate(temperature)
        location_dict[year] = [temperature.mean(), temperature.std()]
    df_year_dict[location] = location_dict
    print(f"{i} %, {location} done!")
    i += 1
print('all location done!')

In [ ]:
df_year = []
for location, dict_1 in df_year_dict.items():
    df_2 = pd.DataFrame(dict_1.values(), index=dict_1.keys(), columns=['AverageTemperature', 'AverageTemperatureUncertainty'])
    df_2['Latitude'] = df[df['location']==location]['Latitude'].unique()[0]
    df_2['Longitude'] = df[df['location']==location]['Longitude'].unique()[0]
    df_2['location'] = location
    df_year.append(df_2)
df_year = pd.concat(df_year)
df_year['year'] = df_year.index 

df_year.to_csv('year.csv', index=False)